# Generate Character from License Plate for Character Recognition

In [7]:
# import dependencies
import os
import yaml
import cv2
import torch

In [8]:
# read data image

data = "place_dataset/data.yaml"

with open(data, 'r') as file:
    dat = yaml.safe_load(file)

train_dir = dat['train'][3:]
test_dir = dat['test'][3:]
val_dir = dat['val'][3:]

# add list of train image
img = [os.path.join(train_dir, i) for i in os.listdir(train_dir)]

# append to test
for i in os.listdir(test_dir):
    img.append(os.path.join(test_dir, i))

# append to valid
for i in os.listdir(val_dir):
    img.append(os.path.join(val_dir, i))

# we can use dir img to process the image 

In [9]:

seg_model =  torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp3/weights/best.pt')

def roi(img, opt: int): # return only one of detect object
    '''
        Extract only Region of Interest (license plate, or region of serial in plate)
        --------------
        img: source of image or array image
        opt: roi (0 for whole plate, 1 for serial)
        
    '''
    # img = cv2.imread(image)
    
    # result = seg_model.predict(img)
    result = seg_model(img, size= 640)
    
    area = []

    for re in result.xyxy:
        for r in re:
            if int(r[-1]) == opt and r[-2] > 0.5: # threshold is r[-2]
                area.append([int(i) for i in r[:4]])
    # for r in result:
    #     for box in r.boxes:
    #         c = box.cls
    #         if c == opt:
    #             area.append([round(i) for i in box.xyxy[0].tolist()])
    return area

Using cache found in /home/nattkorat/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-7-6 Python-3.10.12 torch-2.1.0.dev20230706+cu121 CUDA:0 (NVIDIA GeForce RTX 4070, 12006MiB)

Fusing layers... 
Model summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [13]:
# create database if not exitst
os.makedirs('char_data', exist_ok=True)


In [14]:
save = 0
for i in range(len(img)):
    pic = cv2.imread(img[i])

    info = roi(pic, 1)

    for j in info:
        x, y, x1, y1 = j

        cv2.rectangle(pic, (x, y), (x1, y1), (255, 0, 0), 1)

    cv2.imwrite(f'char_data/{i}_{x}{y}.jpg', pic[y:y1, x:x1])
    save += 1
    print('Saved: ', save, f'char_data/{i}_{x}{y}.jpg')


Saved:  1 char_data/0_26140.jpg
Saved:  2 char_data/1_63274.jpg
Saved:  3 char_data/2_30259.jpg
Saved:  4 char_data/3_31246.jpg
Saved:  5 char_data/4_45264.jpg
Saved:  6 char_data/5_259258.jpg
Saved:  7 char_data/6_259258.jpg
Saved:  8 char_data/7_261197.jpg
Saved:  9 char_data/8_74258.jpg
Saved:  10 char_data/9_300235.jpg
Saved:  11 char_data/10_3177.jpg
Saved:  12 char_data/11_62203.jpg
Saved:  13 char_data/12_62203.jpg
Saved:  14 char_data/13_207261.jpg
Saved:  15 char_data/14_45198.jpg
Saved:  16 char_data/15_59262.jpg
Saved:  17 char_data/16_77271.jpg
Saved:  18 char_data/17_64273.jpg
Saved:  19 char_data/18_89251.jpg
Saved:  20 char_data/19_89251.jpg
Saved:  21 char_data/20_55268.jpg
Saved:  22 char_data/21_32172.jpg
Saved:  23 char_data/22_92291.jpg
Saved:  24 char_data/23_237204.jpg
Saved:  25 char_data/24_61251.jpg
Saved:  26 char_data/25_139280.jpg
Saved:  27 char_data/26_249282.jpg
Saved:  28 char_data/27_39256.jpg
Saved:  29 char_data/28_23238.jpg
Saved:  30 char_data/29_35

In [1]:
import easyocr
import cv2

# Load the English language model
reader = easyocr.Reader(['en'])

# Load an image (replace 'path/to/your/image.jpg' with the path to your image file)
image_path = 'char_data/104_62292.jpg'
image = cv2.imread(image_path)

image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

blurred_image = cv2.GaussianBlur(image, (5, 5), 0)  # Adjust the kernel size as needed

# Apply thresholding to the blurred image
threshold_value = 128  # Adjust this value based on your requirements
_, binary_image = cv2.threshold(blurred_image, threshold_value, 255, cv2.THRESH_BINARY)

# Perform character detection on the image
results = reader.readtext(image)

# # Extract individual characters
# for detection in results:
#     text, bbox = detection[1], detection[0]
#     x, y, w, h = bbox
#     print(bbox)
#     # char_image = image[int(y):int(y+h), int(x):int(x+w)]

# Now 'char_image' contains the cropped image of the detected character
# You can further process 'char_image' or save it to a file

print(results)
cv2.imshow('Detected Character', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[([[0, 0], [538, 0], [538, 170], [0, 170]], '1An:9053', 0.5299874735219187)]
